In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [21]:
nx = 100
ny = nx
nt = 30
dx = 1 / (nx - 1)
dy = dx
dt = 0.0001
alpha = 0.13  # Coeficiente de difusión más alto

x = np.linspace(0, 1, nx)
y = np.linspace(0, 1, ny)

T = np.zeros((nx, ny, nt))
rho = np.ones((nx, ny))  # Densidad constante en toda la placa

X, Y = np.meshgrid(x, y)

# Definir un cilindro con temperatura constante de 0 como distribución inicial de temperatura
cylinder_center_x = 0.5
cylinder_center_y = 0.5
cylinder_radius = 0.3

cylinder_mask = (X - cylinder_center_x) ** 2 + (Y - cylinder_center_y) ** 2 <= cylinder_radius ** 2

T[:, :, 0] = (1 - cylinder_mask) * 100  # Asignar 100 a la placa y 0 al cilindro

# Condiciones de frontera igual a 100
T[0, :, :] = 100
T[nx - 1, :, :] = 100
T[:, 0, :] = 100
T[:, ny - 1, :] = 100

for j in range(nt-1):
    for i in range(1, nx - 1):
        for k in range(1, ny - 1):
            d2T_dx2 = (T[i + 1, k, j] - 2 * T[i, k, j] + T[i - 1, k, j]) / (dx ** 2 * rho[i, k])
            d2T_dy2 = (T[i, k + 1, j] - 2 * T[i, k, j] + T[i, k - 1, j]) / (dy ** 2 * rho[i, k])

            T[i, k, j + 1] = T[i, k, j] + alpha * dt * (d2T_dx2 + d2T_dy2)

    # Condiciones de frontera igual a 100
    T[0, :, j + 1] = 100#50*np.exp(500*(j+1)*dt)
    T[nx - 1, :, j + 1] = 100
    T[:, 0, j + 1] = 100
    T[:, ny - 1, j + 1] = 100



In [48]:
plt.ioff()

fig = plt.figure(figsize=(9,8))
ax = plt.subplot(1,1,1)

ax.set_xlabel('x')
ax.set_ylabel('y')

cont = ax.contourf(T[:,:,0],levels = np.linspace(0,100,11))
cbar = plt.colorbar(cont)
plt.close(fig)

for i in range(nt-1):
    cont = ax.contourf(T[:,:,i],levels = np.linspace(0,100,11))
    fig.savefig(f'figures/{i}.jpg')
    plt.close(fig)